# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/forward_qt.pickle','rb') as f:
    forward_qt = pickle.load(f)

forward_model = load_model('./models/forward_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
forward_df = premier_df[premier_df['포지션']=='공격수']

In [5]:
# 0골인 선수 보정
forward_df['골'] = forward_df['골']+1

# Shoot Accuracy Weighted Goal
forward_df['SAWG'] = (forward_df['경기당 유효 슈팅/90'] / forward_df['경기당 슈팅/90'])*forward_df['골']

In [6]:
# 0도움인 선수 보정

forward_df['도움'] = forward_df['도움']+1

# Pass Accuracy Weighted Assist And Keypass
forward_df['PAWAAK'] = (forward_df['경기당 패스 성공/90'] / forward_df['경기당 패스 시도/90'])*forward_df['도움']+forward_df['키패스/90']

In [7]:
forward_df['태클 시도/90'] = forward_df['태클 성공/90']/forward_df['태클 성공률']

In [8]:
features = ('PAWAAK','SAWG','기회 창출/90','드리블 성공/90','반칙 수/90','크로스 성공률','태클 시도/90','파울 획득')

In [9]:
forward_df.loc[:,features] = forward_qt.transform(forward_df.loc[:,features])

In [10]:
forward_df = forward_df.reset_index(drop=True)

In [11]:
independent_val = ('파울 획득', 'SAWG', '태클 시도/90', '기회 창출/90', '크로스 성공률', 'PAWAAK', '드리블 성공/90')

In [12]:
forward_df['예측PPG']=forward_model.predict(tf.constant(forward_df.loc[:,independent_val].values))

In [13]:
forward_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,SAWG,PAWAAK,태클 시도/90,예측PPG
57,Romelu Lukaku,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,37,1,0,36,10,...,0.80,0.52,1.000000,7.00,0.78,2.03,1.000000,0.000000,0.0,2.626779
63,Harry Kane,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,26,13,0,25,16,...,1.29,0.72,0.996888,7.34,0.95,1.82,0.980797,0.019516,0.0,2.579715
64,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,22,7,0,22,17,...,1.11,1.35,1.000000,3.99,3.67,1.66,0.951460,0.000161,0.0,2.523055
42,Dominic Calvert-Lewin,2021-2022,공격수,에버튼,잉글랜드 프리미어 리그,22,3,0,17,15,...,0.83,0.74,1.000000,6.91,0.83,1.57,0.940629,0.000000,0.0,2.499887
22,Cristiano Ronaldo,2021-2022,공격수,맨 레드,잉글랜드 프리미어 리그,22,10,0,22,13,...,1.58,1.35,1.000000,6.96,2.46,1.80,0.937879,0.008287,0.0,2.493351


In [14]:
forward_min = np.min(forward_df['예측PPG'].tolist())
forward_min

0.7239482402801514

In [15]:
forward_df['WAR'] = forward_df['예측PPG'].map(lambda x : x-forward_min+1)

In [16]:
forward_war = forward_df.sort_values(by='WAR',ascending=False).copy()
forward_war = forward_war.reset_index(drop=True)
forward_war.head(50)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,SAWG,PAWAAK,태클 시도/90,예측PPG,WAR
0,Romelu Lukaku,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,37,1,0,36,10,...,0.52,1.000000,7.00,0.78,2.03,1.000000,0.000000,0.0,2.626779,2.902831
1,Harry Kane,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,26,13,0,25,16,...,0.72,0.996888,7.34,0.95,1.82,0.980797,0.019516,0.0,2.579715,2.855766
2,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,22,7,0,22,17,...,1.35,1.000000,3.99,3.67,1.66,0.951460,0.000161,0.0,2.523055,2.799107
3,Dominic Calvert-Lewin,2021-2022,공격수,에버튼,잉글랜드 프리미어 리그,22,3,0,17,15,...,0.74,1.000000,6.91,0.83,1.57,0.940629,0.000000,0.0,2.499887,2.775939
4,Cristiano Ronaldo,2021-2022,공격수,맨 레드,잉글랜드 프리미어 리그,22,10,0,22,13,...,1.35,1.000000,6.96,2.46,1.80,0.937879,0.008287,0.0,2.493351,2.769403
5,Kai Havertz,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,23,9,0,34,11,...,1.54,1.000000,4.63,2.11,2.05,0.925791,0.001846,0.0,2.467653,2.743705
6,Gabriel Jesus,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,17,2,0,24,11,...,1.03,1.000000,6.18,0.79,1.84,0.923035,0.000000,0.0,2.461816,2.737868
7,Ivan Toney,2021-2022,공격수,브렌트포드,잉글랜드 프리미어 리그,22,2,0,14,4,...,1.13,0.990539,9.84,1.09,1.10,0.923793,0.000000,0.0,2.452018,2.728069
8,Chris Wood,2021-2022,공격수,번리,잉글랜드 프리미어 리그,23,2,0,13,16,...,0.69,0.955912,8.95,1.09,1.28,0.936911,0.000000,0.0,2.431379,2.707431
9,Che Adams,2021-2022,공격수,사우스햄튼,잉글랜드 프리미어 리그,20,2,0,16,7,...,0.65,1.000000,5.99,1.02,1.31,0.896476,0.000000,0.0,2.404590,2.680642


In [17]:
forward_df.query(f'이름 in ["Phil Foden"]')[['이름','WAR']]

,이름,WAR
18,Phil Foden,1.443541


In [18]:
forward_war.loc[:,features] = forward_qt.inverse_transform(forward_war.loc[:,features])

In [19]:
forward_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(40)

,이름,시즌,골,도움,WAR
0,Romelu Lukaku,2021-2022,37,1,2.902831
1,Harry Kane,2021-2022,26,13,2.855766
2,Heung-Min Son,2021-2022,22,7,2.799107
3,Dominic Calvert-Lewin,2021-2022,22,3,2.775939
4,Cristiano Ronaldo,2021-2022,22,10,2.769403
5,Kai Havertz,2021-2022,23,9,2.743705
6,Gabriel Jesus,2021-2022,17,2,2.737868
7,Ivan Toney,2021-2022,22,2,2.728069
8,Chris Wood,2021-2022,23,2,2.707431
9,Che Adams,2021-2022,20,2,2.680642
